In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas
import math
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dropout
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

In [2]:
data_x = np.loadtxt('gesture_input_x_v3.csv', dtype=float)
data_y = np.loadtxt('gesture_input_y_v3.csv', dtype=float)

data_x.shape, data_y.shape

((194, 150), (194, 4))

In [3]:
# fix random seed for reproducibility
np.random.seed(7)

In [4]:
# split into train and test sets
train_size = int(len(data_x) * 0.6)
validation_size = int(len(data_x) * 0.2)
test_size = len(data_x) - validation_size - train_size
train_size, validation_size, test_size

(116, 38, 40)

In [5]:
train_x, vaildation_x, test_x = data_x[0:train_size,:], data_x[train_size:train_size+validation_size,:], data_x[train_size+validation_size:,:]
train_y, vaildation_y, test_y = data_y[0:train_size,:], data_y[train_size:train_size+validation_size,:], data_y[train_size+validation_size:,:]

In [6]:
train_x.shape, train_y.shape, vaildation_x.shape, vaildation_y.shape, test_x.shape, test_y.shape

((116, 150), (116, 4), (38, 150), (38, 4), (40, 150), (40, 4))

In [7]:
# reshape input to be [samples, time steps, features]
train_x = np.reshape(train_x, (train_x.shape[0], 1, train_x.shape[1]))
vaildation_x = np.reshape(vaildation_x, (vaildation_x.shape[0], 1, vaildation_x.shape[1]))
test_x = np.reshape(test_x, (test_x.shape[0], 1, test_x.shape[1]))

In [17]:
regularizer = tf.contrib.layers.l2_regularizer(scale=0.001)

In [18]:
# create and fit the LSTM network
model = Sequential()
model.add(LSTM(150, activation=tf.nn.tanh, kernel_regularizer=regularizer, input_shape=(1, 150)))
model.add(Dense(100, activation=tf.nn.relu, kernel_regularizer=regularizer))
model.add(Dense(50, activation=tf.nn.relu, kernel_regularizer=regularizer))
model.add(Dense(4, activation=tf.nn.softmax))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_2 (LSTM)                (None, 150)               180600    
_________________________________________________________________
dense_5 (Dense)              (None, 100)               15100     
_________________________________________________________________
dense_6 (Dense)              (None, 50)                5050      
_________________________________________________________________
dense_7 (Dense)              (None, 4)                 204       
Total params: 200,954
Trainable params: 200,954
Non-trainable params: 0
_________________________________________________________________


In [19]:
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])
model.fit(train_x, train_y, epochs=100, batch_size=10, validation_data=(vaildation_x, vaildation_y), verbose=1)


Train on 116 samples, validate on 38 samples
Epoch 1/100
116/116 [==============================] - 1s 10ms/sample - loss: 0.3529 - acc: 0.3017 - val_loss: 0.3465 - val_acc: 0.3158
Epoch 2/100
116/116 [==============================] - 0s 738us/sample - loss: 0.3096 - acc: 0.3966 - val_loss: 0.3059 - val_acc: 0.4474
Epoch 3/100
116/116 [==============================] - 0s 714us/sample - loss: 0.2706 - acc: 0.4397 - val_loss: 0.2697 - val_acc: 0.5526
Epoch 4/100
116/116 [==============================] - 0s 686us/sample - loss: 0.2408 - acc: 0.5086 - val_loss: 0.2450 - val_acc: 0.5263
Epoch 5/100
116/116 [==============================] - 0s 658us/sample - loss: 0.2240 - acc: 0.4655 - val_loss: 0.2329 - val_acc: 0.5263
Epoch 6/100
116/116 [==============================] - 0s 727us/sample - loss: 0.2016 - acc: 0.4914 - val_loss: 0.2166 - val_acc: 0.5000
Epoch 7/100
116/116 [==============================] - 0s 608us/sample - loss: 0.1845 - acc: 0.5345 - val_loss: 0.2063 - val_acc: 0.55

116/116 [==============================] - 0s 887us/sample - loss: 0.0920 - acc: 0.6897 - val_loss: 0.1166 - val_acc: 0.7105
Epoch 61/100
116/116 [==============================] - 0s 720us/sample - loss: 0.0914 - acc: 0.6638 - val_loss: 0.1073 - val_acc: 0.7105
Epoch 62/100
116/116 [==============================] - 0s 649us/sample - loss: 0.0944 - acc: 0.6724 - val_loss: 0.1296 - val_acc: 0.7105
Epoch 63/100
116/116 [==============================] - 0s 665us/sample - loss: 0.0999 - acc: 0.6207 - val_loss: 0.1243 - val_acc: 0.7105
Epoch 64/100
116/116 [==============================] - 0s 694us/sample - loss: 0.0944 - acc: 0.6638 - val_loss: 0.1179 - val_acc: 0.7105
Epoch 65/100
116/116 [==============================] - 0s 650us/sample - loss: 0.0948 - acc: 0.6724 - val_loss: 0.1098 - val_acc: 0.7105
Epoch 66/100
116/116 [==============================] - 0s 699us/sample - loss: 0.0944 - acc: 0.6552 - val_loss: 0.1269 - val_acc: 0.7105
Epoch 67/100
116/116 [=========================

In [ ]:
# model.evaluate(test_x, test_y)

In [23]:
prediction = model.predict(test_x.reshape(test_x.shape[0], 1, 150))

In [36]:
t = [np.greater(i, 0.5) for i in prediction]
for i in t:
    gesture = np.where(i)
    print(i, "None" if len(gesture[0]) == 0 else gesture[0][0])

[False False False  True] 3
[False False False  True] 3
[ True False False False] 0
[ True False False False] 0
[False  True False False] 1
[False False False False] None
[False False False  True] 3
[False  True False False] 1
[False False  True False] 2
[False False False False] None
[False False False False] None
[False False False False] None
[ True False False False] 0
[False False False  True] 3
[False False False  True] 3
[ True False False False] 0
[False  True False False] 1
[False False  True False] 2
[False False False  True] 3
[False  True False False] 1
[ True False False False] 0
[False False False  True] 3
[ True False False False] 0
[False False False  True] 3
[ True False False False] 0
[False  True False False] 1
[ True False False False] 0
[False False False  True] 3
[ True False False False] 0
[ True False False False] 0
[False False False  True] 3
[ True False False False] 0
[False  True False False] 1
[False False False  True] 3
[False False False  True] 3
[ True F

In [37]:
model.save("gesture_lstm_v4.h5")